### **Representaciones distribuidas**

Las representaciones distribuidas han transformado el campo del procesamiento de lenguaje natural (NLP) y el aprendizaje automático. A diferencia de enfoques basados en codificaciones locales, como one-hot encoding, que generan vectores dispersos de alta dimensión con valores mayoritariamente en cero, las representaciones distribuidas aprenden vectores densos de dimensión fija en un espacio continuo. 

Gracias a la hipótesis distributiva, según la cual palabras con contextos similares comparten significados próximos, estos métodos capturan relaciones semánticas complejas, optimizan el uso de memoria y facilitan el aprendizaje de significados de palabras nuevas o raras.


#### **Características principales**

Veamos algunos características de estos métodos:

- A diferencia de las representaciones locales, las distribuidas pueden capturar relaciones complejas entre palabras, como sinónimos, antónimos o términos que suelen aparecer en contextos similares.

- Al representar palabras como vectores de tamaño fijo en un espacio continuo, se reduce la dimensionalidad del problema comparado con métodos de representación más simples pero de alta dimensionalidad, como el one-hot encoding.

- Estos modelos pueden generalizar para entender palabras nuevas o raras a partir de sus componentes (por ejemplo, entender palabras compuestas a partir de los significados de sus partes).

**Ejemplos y modelos**

- Word2Vec: Probablemente el ejemplo más conocido de representaciones distribuidas. Word2Vec utiliza redes neuronales para aprender representaciones vectoriales de palabras a partir de grandes conjuntos de datos de texto. Ofrece dos arquitecturas principales: CBOW (Continuous Bag of Words) y Skip-gram, cada una diseñada para aprender representaciones que predigan palabras en función de sus contextos o viceversa.

- GloVe (Global Vectors for Word Representation): Un modelo que aprende representaciones de palabras a partir de las estadísticas co-ocurrenciales de palabras en un corpus. La idea es que las relaciones semánticas entre palabras pueden ser capturadas observando qué tan frecuentemente aparecen juntas en un gran corpus.

- Embeddings contextuales: Modelos más recientes como ELMo, BERT y GPT ofrecen una evolución de las representaciones distribuidas, generando vectores de palabras que varían según el contexto en el que aparecen, lo que permite capturar usos y significados múltiples de una misma palabra dependiendo de la oración en la que se encuentre.

### **Embeddings de palabras**

Los embeddings de palabras son representaciones vectoriales densas y de baja dimensión de palabras, diseñadas para capturar el significado semántico, sintáctico y relaciones entre ellas. A diferencia de las representaciones de texto más antiguas, como el one-hot encoding, que son dispersas (la mayoría de los valores son cero) y de alta dimensión, los embeddings de palabras se representan en un espacio vectorial continuo donde palabras con significados similares están ubicadas cercanamente en el espacio vectorial.

**Características de los embeddings de palabras**

- Cada palabra se representa como un vector denso, lo que significa que cada dimensión tiene un valor real, a diferencia de los vectores dispersos de otras técnicas de representación.

- Los embeddings generalmente tienen un tamaño de dimensión fijo y relativamente pequeño (por ejemplo, 100, 200, 300 dimensiones) independientemente del tamaño del vocabulario.

- Estos vectores intentan capturar el contexto y el significado de una palabra, no solo su presencia o ausencia. Palabras que se usan en contextos similares tendrán embeddings similares.

- Pueden ayudar a los modelos de aprendizaje automático a generalizar mejor a palabras no vistas durante el entrenamiento, dado que las palabras con significados similares se mapean a puntos cercanos en el espacio vectorial.


En 2013, un trabajo fundamental de Mikolov [Efficient Estimationof Word Representations in Vector Space](https://arxiv.org/abs/1301.3781) demostraron que su modelo de representación de palabras basado en una red neuronal conocido como `Word2vec`, basado en la `similitud distributiva`, puede capturar relaciones de analogía de palabras como: 

$$King - Man + Woman \approx Queen$$

Conceptualmente, Word2vec toma un gran corpus de texto como entrada y "aprende" a representar las palabras en un espacio vectorial común en función de los contextos en los que aparecen en el corpus.


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from collections import Counter
import random

# 1. Corpus de ejemplo
sentences = [
    ['king', 'queen', 'man', 'woman'],
    ['man', 'king', 'prince'],
    ['woman', 'queen', 'princess'],
    ['prince', 'son', 'king'],
    ['princess', 'daughter', 'queen']
]

# 2. Construir vocabulario
words = [w for sent in sentences for w in sent]
vocab = {w: i for i, (w, _) in enumerate(Counter(words).items())}
inv_vocab = {i: w for w, i in vocab.items()}
V = len(vocab)

# 3. Generar pares (target, context)
window_size = 2
pairs = []
for sent in sentences:
    indices = [vocab[w] for w in sent]
    for center_pos, center in enumerate(indices):
        for w in range(-window_size, window_size + 1):
            context_pos = center_pos + w
            if context_pos < 0 or context_pos >= len(indices) or context_pos == center_pos:
                continue
            pairs.append((center, indices[context_pos]))

# 4. Modelo Skip‑gram simple
class SkipGram(nn.Module):
    def __init__(self, vocab_size, emb_dim):
        super().__init__()
        self.u_embeddings = nn.Embedding(vocab_size, emb_dim, sparse=True)
        self.v_embeddings = nn.Embedding(vocab_size, emb_dim, sparse=True)

    def forward(self, u, v):
        u_emb = self.u_embeddings(u)       # (batch, emb_dim)
        v_emb = self.v_embeddings(v)       # (batch, emb_dim)
        score = torch.mul(u_emb, v_emb).sum(dim=1)  # dot product
        return score

# 5. Entrenamiento
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
modelo = SkipGram(vocab_size=V, emb_dim=50).to(device)
optimizer = optim.SGD(modelo.parameters(), lr=0.01)
criterion = nn.BCEWithLogitsLoss()

# función para muestrear negativos
def negative_sampling(batch_size, K):
    negs = torch.randint(0, V, (batch_size * K,), device=device)
    return negs

epochs = 200
K = 5  # número de negativos por par positivo

for epoch in range(epochs):
    total_loss = 0
    random.shuffle(pairs)
    for center, context in pairs:
        u = torch.tensor([center], dtype=torch.long, device=device)
        v_pos = torch.tensor([context], dtype=torch.long, device=device)
        # positivos → etiqueta 1
        pos_score = modelo(u, v_pos)
        pos_loss  = criterion(pos_score, torch.ones_like(pos_score))
        # negativos → etiqueta 0
        neg_v = negative_sampling(batch_size=1, K=K)
        neg_score = modelo(u.repeat(K), neg_v)
        neg_loss  = criterion(neg_score, torch.zeros_like(neg_score))
        # retroprop y optimización
        loss = pos_loss + neg_loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    if (epoch+1) % 50 == 0:
        print(f"Epoca {epoch+1}/{epochs}, Loss: {total_loss/len(pairs):.4f}")

# 6. Obtener embeddings y analogías
embeddings = modelo.u_embeddings.weight.data.cpu()

def most_similar(word, topn=3):
    idx = vocab[word]
    query = embeddings[idx]
    # similitud coseno
    cos = torch.nn.functional.cosine_similarity(query.unsqueeze(0), embeddings)
    best = torch.topk(cos, topn+1).indices.tolist()[1:]  # excluye la misma palabra
    return [(inv_vocab[i], cos[i].item()) for i in best]

print("\nSimilares a 'king':", most_similar('king'))
print("Analogía (king - man + woman):")
# v = king - man + woman
vec = embeddings[vocab['king']] - embeddings[vocab['man']] + embeddings[vocab['woman']]
cos = torch.nn.functional.cosine_similarity(vec.unsqueeze(0), embeddings)
best = torch.topk(cos, 3).indices.tolist()
for i in best:
    print(f"  {inv_vocab[i]}: {cos[i].item():.3f}")


#### **Embeddings de palabras pre-entrenadas**

Podemos embeddings de Word2vec previamente entrenadas y buscar las palabras más similares (clasificadas por similitud de coseno) a una palabra determinada. 

Tomemos un ejemplo de un modelo word2vec previamente entrenado y cómo podemos usarlo para buscar la mayoría de las palabras similares. Usaremos los embeddings de vectores de Google News. https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM

Se pueden encontrar algunos otros modelos de embeddings de palabras previamente entrenados y detalles sobre los medios para acceder a ellos a través de gensim en: https://github.com/RaRe-Technologies/gensim-data

El código que sigue cubre los pasos clave. Aquí encontramos las palabras que semánticamente son más similares a la palabra "beautiful"; la última línea devuelve el vector de embeddings de la palabra " beautiful ":

In [ ]:
!python -m spacy download en_core_web_md


In [ ]:
import spacy
from heapq import nlargest
import numpy as np

# 1. Carga el modelo con vectores de tamaño 300
nlp = spacy.load("en_core_web_md")

# 2. Obtén el vector de "beautiful"
v_beautiful = nlp.vocab["beautiful"].vector

# 3. Calcula similitud coseno con todo el vocabulario
sims = []
norm_beautiful = np.linalg.norm(v_beautiful)
for lex in nlp.vocab:
    if lex.has_vector and not lex.is_stop and lex.is_alpha:
        v = lex.vector
        score = np.dot(v_beautiful, v) / (norm_beautiful * np.linalg.norm(v))
        sims.append((lex.text, float(score)))

# 4. Top‑10 más similares
top10 = nlargest(10, sims, key=lambda x: x[1])
print("Top 10 palabras similares a 'beautiful':")
for w, score in top10:
    print(f"  {w:15s} → {score:.4f}")

# 5. Inspecciona el vector de 'beautiful'
print("\nDimensión del vector:", v_beautiful.shape)
print("Primeros 10 valores:", v_beautiful[:10])


Dos cosas a tener en cuenta al utilizar modelos previamente entrenados:

* Los tokens/palabras siempre están en minúsculas. Si una palabra no está en el vocabulario, el modelo genera una excepción.
* Por lo tanto, siempre es una buena idea encapsular esas declaraciones en bloques `try/except`.

### **Representaciones distribuidas avanzadas**

En la evolución de las representaciones distribuidas, surgieron cuatro líneas de investigación fundamentales que atendieron necesidades concretas: mejorar la generalización, reducir la carga computacional, incorporar conocimiento morfológico y compartir representaciones entre tareas. A continuación describimos cada enfoque, su motivación y la fórmula matemática que explica su funcionamiento.

#### **Neural network language model (NNLM)**

**Contexto**  
Antes de 2003, las representaciones basadas en conteo (coocurrencias, SVD, PMI) carecían de capacidad para modelar interacciones no lineales. Bengio et al. propusieron entrenar de forma conjunta los embeddings y una red feed‑forward que, para una ventana de contexto fija $(w_{t-n+1},\dots,w_{t-1})$, estimara la probabilidad de la siguiente palabra $w_t$.

**Ecuación clave**  

$$
P(w_t \mid w_{t-n+1},\dots,w_{t-1})
=
\frac{\exp\bigl(u_{w_t}^{\!\top}\,f\bigl(W\,[\,e_{w_{t-n+1}},\dots,e_{w_{t-1}}]\bigr)\bigr)}
     {\sum_{w\in V}\exp\bigl(u_{w}^{\!\top}\,f\bigl(W\,[\,e_{w_{t-n+1}},\dots,e_{w_{t-1}}]\bigr)\bigr)},
$$  

donde:  
- $e_{w}\in\mathbb{R}^d$ es el embedding de cada palabra de contexto.  
- $W$ y $u_w$ son parámetros entrenables de la red.  
- $f$ es una función oculta no lineal (por ejemplo, tanh).  
- El denominador softmax normaliza sobre todo el vocabulario $V$.  

Este planteamiento mejoró la predicción de secuencias y sentó las bases de los embeddings neuronales.


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# 1. Parámetros del modelo
vocab = ["I", "like", "dogs", "cats", "and"]
word2idx = {w: i for i, w in enumerate(vocab)}
V = len(vocab)         # tamaño del vocabulario
d = 10                 # dimensión de embeddings
n = 2                  # tamaño de la ventana de contexto
hidden_dim = 20        # dimensión de la capa oculta

# 2. Definición del modelo NNLM
class NNLM(nn.Module):
    def __init__(self, vocab_size, emb_dim, context_size, hidden_dim):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, emb_dim)
        self.linear1 = nn.Linear(context_size * emb_dim, hidden_dim)
        self.tanh = nn.Tanh()
        self.linear2 = nn.Linear(hidden_dim, vocab_size)
    
    def forward(self, context_idxs):
        """
        context_idxs: tensor Long de forma (batch_size, context_size)
        Salida: distribuciones de probabilidad Softmax de forma (batch_size, vocab_size)
        """
        emb = self.emb(context_idxs)                        # (batch, context_size, emb_dim)
        emb = emb.view(emb.size(0), -1)                     # (batch, context_size*emb_dim)
        h = self.tanh(self.linear1(emb))                    # (batch, hidden_dim)
        logits = self.linear2(h)                            # (batch, vocab_size)
        probs = torch.softmax(logits, dim=1)                # (batch, vocab_size)
        return probs

# 3. Crear instancia del modelo y optimizador
modelo = NNLM(V, d, n, hidden_dim)
optimizer = optim.SGD(modelo.parameters(), lr=0.1)
criterion = nn.NLLLoss()

# 4. Ejemplo de un batch de entrenamiento
# Contexto: ["I","like"] predice target "dogs"
context_batch = torch.tensor([[word2idx["I"], word2idx["like"]]], dtype=torch.long)
target_batch = torch.tensor([word2idx["dogs"]], dtype=torch.long)

# Forward pass
probs = modelo(context_batch)                            # (1, V)
log_probs = torch.log(probs)

# Cálculo de la pérdida y backward
loss = criterion(log_probs, target_batch)
optimizer.zero_grad()
loss.backward()
optimizer.step()

# 5. Salidas
print("Distribución P(w_t|context=['I','like']):")
for w, idx in word2idx.items():
    print(f"  P({w}) = {probs[0, idx].item():.4f}")
print(f"\nPérdida (NLLLoss): {loss.item():.4f}")



#### **C&W Model (Collobert & Weston)**

**Contexto**  
La necesidad de compartir embeddings entre múltiples tareas (clasificación, análisis sintáctico, etc.) y de evitar el alto coste de la normalización softmax condujo a Collobert y Weston (2008) a diseñar un esquema de aprendizaje por ranking con redes convolucionales.

**Ecuación clave**  

Definiendo una ventana real $x^+$ y variantes corruptas $x^-$, el objetivo hinge es:  
$$
\mathcal{L}_{\mathrm{C\&W}}
=
\sum_{x^+}\sum_{x^-}
\max\bigl(\,0,\,1 - s(x^+) + s(x^-)\bigr),
$$  
donde $s(x)$ es una puntuación escalar obtenida aplicando convoluciones y capas densas sobre la secuencia de embeddings concatenados. Esta pérdida garantiza que los ejemplos reales obtengan siempre al menos un margen de 1 frente a los negativos, acelerando el entrenamiento y facilitando el multitasking.




In [ ]:
import numpy as np

# 1. Parámetros básicos
vocab = ["I", "love", "cats", "dogs", "and"]
word2idx = {w: i for i, w in enumerate(vocab)}
V = len(vocab)
d = 50             # dimensión de embeddings
window_size = 3    # tamaño de la ventana
np.random.seed(42)

# 2. Inicializar embeddings y vector de scoring
E = np.random.randn(V, d)                     # tabla de embeddings aleatorios
w_score = np.random.randn(window_size * d)    # vector para scoring

# 3. Función de scoring s(x)
def score(window):
    # Concatena embeddings de la ventana y aplica el scoring lineal
    concat_emb = np.hstack([E[word2idx[w]] for w in window])
    return float(concat_emb.dot(w_score))

# 4. Ventana real (x+) y corrupta (x-)
x_pos = ["I", "love", "cats"]
x_neg = ["I", "dogs", "cats"]  # corrompemos la palabra central

# 5. Cálculo de scores y pérdida hinge
s_pos = score(x_pos)
s_neg = score(x_neg)
margin = 1.0
loss = max(0.0, 1 - s_pos + s_neg)

# 6. Resultados
print("Ventana real:", x_pos, "→ score:", f"{s_pos:.4f}")
print("Ventana corrupta:", x_neg, "→ score:", f"{s_neg:.4f}")
print("C&W Hinge Loss:", f"{loss:.4f}")


#### **CBOW y Skip‑gram**

**Contexto**  
Word2Vec introdujo dos tareas inversas que, combinadas con técnicas de muestreo, hicieron posible entrenar embeddings de alta calidad en vocabularios enormes.

- **CBOW** promedia el contexto para predecir la palabra central.
- **Skip‑gram** invierte la tarea, usando la palabra central para predecir su contexto.

**Ecuación clave (Negative Sampling)**  
En lugar de softmax completo, usan:
$$
\mathcal{L}_{\text{NS}}
=
-\log\sigma\bigl(u_{w_t}^{\!\top}v_C\bigr)
\;-\;
\sum_{i=1}^k
\mathbb{E}_{w_i\sim P_n}
\bigl[\log\sigma\bigl(-u_{w_i}^{\!\top}v_C\bigr)\bigr],
$$
donde  
- $v_C$ es el embedding del contexto (o de la palabra central).  
- $\sigma(x)=1/(1+e^{-x})$.  
- $P_n$ es una distribución de ruido habitual (frecuencias a la 3/4).  
- $k$ es el número de muestras negativas.  

Gracias a esta reformulación, la complejidad por ejemplo pasó de $O(|V|)$ a $O(k)$.




In [ ]:
import numpy as np

# Vocabulario y mapeo
vocab = ["I", "like", "NLP", "and", "AI"]
word2idx = {w: i for i, w in enumerate(vocab)}
V = len(vocab)
d = 5  # dimensión de embeddings

# Inicializar embeddings de entrada (W_in) y salida (W_out)
np.random.seed(0)
W_in = np.random.randn(V, d)
W_out = np.random.randn(V, d)

# Distribución de ruido unigram^3/4
freqs = np.array([1, 2, 1, 1, 1], dtype=np.float32)
p_noise = freqs**0.75
p_noise /= p_noise.sum()

# Función sigma
sigma = lambda x: 1 / (1 + np.exp(-x))

# --- CBOW con Negative Sampling ---
context = ["I", "NLP", "and", "AI"]
target = "like"
# Embedding del contexto
v_C = np.mean([W_in[word2idx[w]] for w in context], axis=0)  # (d,)
# Score positivo
u_t = W_out[word2idx[target]]
pos_score = sigma(u_t.dot(v_C))
# Muestras negativas
k = 2
neg_idxs = np.random.choice(V, size=k, p=p_noise)
neg_scores = sigma(-W_out[neg_idxs].dot(v_C))
# Pérdida CBOW
cbow_loss = -np.log(pos_score) - np.sum(np.log(neg_scores))

# Mostrar resultados CBOW
print("CBOW con Negative Sampling")
print(f"Contexto: {context} → Target: '{target}'")
print(f"Positive idx: {word2idx[target]}, Negative idxs: {neg_idxs.tolist()}")
print(f"pos_score: {pos_score:.4f}")
print(f"neg_scores: {[round(s,4) for s in neg_scores]}")
print(f"CBOW Loss: {cbow_loss:.4f}\n")

# --- Skip-gram con Negative Sampling ---
center = "like"
contexts = ["I", "NLP"]
v_center = W_in[word2idx[center]]
pos_scores_sg = sigma(np.array([W_out[word2idx[w]].dot(v_center) for w in contexts]))
# Muestras negativas para cada contexto
neg_idxs_sg = np.random.choice(V, size=(len(contexts), k), p=p_noise)
neg_scores_sg = sigma(-np.array([W_out[neg_idxs_sg[i]].dot(v_center) for i in range(len(contexts))]))
# Pérdida Skip-gram
skip_loss = -np.sum(np.log(pos_scores_sg)) - np.sum(np.log(neg_scores_sg))

# Mostrar resultados Skip-gram
print("Skip-gram con Negative Sampling")
print(f"Center: '{center}' → Contextos: {contexts}")
print(f"Positive idxs: {[word2idx[w] for w in contexts]}, Negative idxs:\n  {neg_idxs_sg.tolist()}")
print(f"pos_scores: {[round(s,4) for s in pos_scores_sg]}")
print(f"neg_scores:\n  {[[round(s,4) for s in row] for row in neg_scores_sg]}")
print(f"Skip-gram Loss: {skip_loss:.4f}")



#### **Método híbrido caracter‑palabra**

**Contexto**  
Las lenguas ricas en morfología y el problema de las palabras OOV inspiraron la combinación de embeddings de caracteres y de palabra. Mediante CNNs o LSTMs se aprenden vectores de caracteres o n‑gramas, que luego se fusionan con el embedding de la palabra completa.

**Ecuación clave**  
Para un término $w$ compuesto por $L$ n‑gramas de caracteres $c_i$, la representación final es:  
$$
v_w
=
W_1\,e_w
\;+\,
W_2\,
\Bigl(\tfrac{1}{L}\sum_{i=1}^L c_i\Bigr),
$$  
con $W_1$ y $W_2$ como matrices de combinación entrenables. FastText ejemplifica esta idea descomponiendo cada palabra en todos sus n‑gramas y sumando sus embeddings, logrando robustez frente a variaciones ortográficas y morfológicas.


In [ ]:
import numpy as np

# 1. Definición de palabras de ejemplo
words = ["unbelievable", "running"]

# 2. Función para extraer n‑gramas de caracteres
def extract_ngrams(word, min_n=3, max_n=6):
    word_aug = f"<{word}>"
    ngrams = []
    for n in range(min_n, max_n + 1):
        for i in range(len(word_aug) - n + 1):
            ngrams.append(word_aug[i:i+n])
    return ngrams

# 3. Construir vocabulario de caracteres (n‑gramas)
all_ngrams = set(ng for w in words for ng in extract_ngrams(w))
char_vocab = {ng: i for i, ng in enumerate(sorted(all_ngrams))}

# 4. Inicializar embeddings aleatorios y matrices de combinación
emb_dim = 50
np.random.seed(0)
# Embedding de la "palabra completa"
word_emb_table = {w: np.random.randn(emb_dim) for w in words}
# Embedding de cada n‑grama
char_emb_table = np.random.randn(len(char_vocab), emb_dim)
# Matrices W1 y W2 entrenables
W1 = np.random.randn(emb_dim, emb_dim)
W2 = np.random.randn(emb_dim, emb_dim)

# 5. Cálculo de embeddings híbridos según la ecuación v_w = W1 e_w + W2 (1/L sum c_i)
hybrid_embeddings = {}
for w in words:
    e_w = word_emb_table[w]  # embedding de la palabra
    ngrams = extract_ngrams(w)
    c_idxs = [char_vocab[ng] for ng in ngrams]
    e_c = char_emb_table[c_idxs].mean(axis=0)  # promedio de embeddings de n‑gramas
    v_w = W1 @ e_w + W2 @ e_c
    hybrid_embeddings[w] = v_w

# 6. Mostrar resultados
for w, vec in hybrid_embeddings.items():
    print(f"'{w}' → embedding híbrido (primeros 5 valores): {vec[:5]}")



### **Representaciones distribuidas de frases**

Las representaciones distribuidas de frases constituyen un paso intermedio entre los embeddings de palabras y los de documentos completos. Si bien las palabras aportan unidades atómicas de significado, las frases introducen composiciones semánticas y relaciones sintácticas complejas que los modelos deben capturar para tareas como análisis de sentimientos, clasificación de oraciones y recuperación de información. 

En este contexto, las dos familias clásicas de métodos son los basados en Bag‑of‑Words (BoW), que extienden de forma lineal la lógica de los embeddings de palabras, y los basados en autoencoders, que emplean arquitecturas neuronales para codificar y decodificar la secuencia completa.

#### **Basadas en bolsas de palabras**

Los métodos Bag‑of‑Words para frases aprovechan la disponibilidad de embeddings de palabras preentrenados y definen la representación de una frase como agregación de los vectores de sus tokens. La simplicidad y eficiencia de estos esquemas han garantizado su vigencia en entornos con grandes volúmenes de datos.

**Promedio simple de embeddings**

Sea una frase $S = (w_1, w_2, \,\dots\,, w_n)$ con embeddings de palabras $v_i = e_{w_i} \in \mathbb{R}^d$. El promedio simple define:

$$
  v_{S} = \frac{1}{n} \sum_{i=1}^n v_i.
$$

Este método asume que cada token contribuye de igual manera a la semántica global. A pesar de ignorar el orden y las interacciones no lineales, su bajo coste computacional $O(n\times d)$ lo convierte en una opción popular, especialmente para tareas de similitud semántica donde la magnitud de la frase importa más que su estructura interna.

**Promedios ponderados y smooth inverse frequency (SIF)**

Para reflejar la relevancia diferencial de cada palabra, se introducen pesos basados en estadísticas de corpus. El esquema SIF (Arora et al., 2017) propone:

$$
  \alpha_i = \frac{a}{a + p(w_i)},
  \quad
  v_{S} = \frac{1}{\sum_{i=1}^n \alpha_i} \sum_{i=1}^n \alpha_i v_i,
$$

donde $p(w_i)$ es la frecuencia de $w_i$ en un gran corpus y $a$ un parámetro de suavizado (por ejemplo, $10^{-3}$). Después se suprime la primera componente principal $u$ de todo el conjunto de vectores de frases:

$$
  v_{S}' = v_{S} - u\bigl(u^\top v_{S}\bigr).
$$

La eliminación de la componente dominante reduce el efecto de palabras muy frecuentes y mejora la discriminación entre frases semánticamente similares.

**Representación TF‑IDF ponderada**

Como alternativa, se utiliza el peso TF‑IDF de cada término:

$$
  \alpha_i = \mathrm{tf}(w_i, S) \times \log\frac{N}{\mathrm{df}(w_i)},
  \quad
  v_{S} = \frac{1}{\sum_{i=1}^n \alpha_i} \sum_{i=1}^n \alpha_i v_i,
$$

con $N$ el número total de documentos y $\mathrm{df}(w_i)$ la cantidad de documentos que contienen $w_i$. Este esquema potencia términos distintivos y atenúa los muy comunes.

**Max‑pooling, min‑pooling y concatenaciones**

Otros esquemas no lineales incluyen:

- **Max‑pooling:** $v_S(j) = \max_{1\le i\le n} v_i(j)$ para cada dimensión $j$.
- **Min‑pooling:** $v_S(j) = \min_{1\le i\le n} v_i(j)$.
- **Concatenación:** combinación de $\langle\mathrm{mean},\mathrm{max},\mathrm{min}\rangle$ generando un vector de dimensión $3d$.

Aunque capturan características de forma más rica, incrementan la dimensionalidad y el coste de almacenamiento.

**Interpretación empírica y aplicaciones**

Los métodos BoW han mostrado:

- **Buen desempeño en similitud semántica:** la correlación coseno entre $v_S$ y otro vector de referencia se alinea con la percepción humana de similitud.
- **Eficiencia en recuperación de información:** indexación de vectores en aproximaciones de "nearest neighbor" para búsqueda rápida.

No obstante, son sensibles a la presencia de stopwords y carecen de modelado de negaciones y dependencia sintáctica.

**Limitaciones sintácticas y semánticas**

| Limitación                  | Descripción                                                                         |
|-----------------------------|-------------------------------------------------------------------------------------|
| Invarianza al orden         | No distingue "rojo coche rápido" de "rápido coche rojo"                           |
| Escasa interacción léxica   | No captura modismos ni expresiones idiomáticas                                      |
| Vulnerable a palabras vacías| Stopwords frecuentes pueden dominar la media sin corrección de ponderación adecuada |
| Sin dependencia gramatical  | No modela relaciones sujeto-verbo u objetos directos e indirectos                    |

En entornos donde la sintaxis sea clave (por ejemplo, detección de sarcasmo o análisis profundo de relaciones), se prefieren métodos más complejos.




In [ ]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA

# 1. Definir frases de ejemplo
sentences = [
    "The quick brown fox jumps over the lazy dog",
    "A fast brown fox leaps over lazy dogs in the park",
    "Deep learning models require large amounts of data",
    "Machine learning and deep learning are subfields of AI",
    "Natural language processing enables computers to understand text"
]

# 2. Tokenización básica (split + limpiar no alfabéticos)
tokenized = []
for sent in sentences:
    tokens = [w.lower() for w in sent.split() if w.isalpha()]
    tokenized.append(tokens)

# 3. Construir vocabulario de tokens únicos
all_tokens = [tok for sent in tokenized for tok in sent]
vocab = sorted(set(all_tokens))
word2idx = {w:i for i,w in enumerate(vocab)}
V = len(vocab)
d = 50  # dimensión de embeddings

# 4. Inicializar embeddings aleatorios
np.random.seed(0)
E = np.random.randn(V, d)

# 5. Frecuencias de palabras p(w) para SIF
word_counts = {w: all_tokens.count(w) for w in vocab}
total = sum(word_counts.values())

# 6. Función auxiliar: promedio simple
def avg_embedding(tokens):
    vecs = np.array([E[word2idx[t]] for t in tokens])
    return vecs.mean(axis=0)

# 7. Promedio simple para cada frase
simple_avg = [avg_embedding(sent) for sent in tokenized]

# 8. SIF embeddings
a = 1e-3
sif_vecs = []
for sent in tokenized:
    weights = [a / (a + word_counts[t]/total) for t in sent]
    vecs = np.array([E[word2idx[t]] for t in sent])
    weighted = (vecs * np.array(weights)[:,None]).sum(axis=0) / sum(weights)
    sif_vecs.append(weighted)

# 9. Eliminar componente principal de SIF
pca = PCA(n_components=1)
pca.fit(np.vstack(sif_vecs))
u = pca.components_[0]
sif_processed = [v - u*(u.dot(v)) for v in sif_vecs]

# 10. TF-IDF ponderado
tfidf = TfidfVectorizer(lowercase=True, token_pattern=r"(?u)\b[a-zA-Z]+\b")
X = tfidf.fit_transform(sentences)
feat = tfidf.get_feature_names_out()
tfidf_embs = []
for i, sent in enumerate(tokenized):
    weights, vecs = [], []
    for t in sent:
        if t in feat:
            idx = np.where(feat==t)[0][0]
            w = X[i, idx]
            weights.append(w)
            vecs.append(E[word2idx[t]])
    weights = np.array(weights)
    vecs = np.array(vecs)
    tfidf_embs.append((vecs*weights[:,None]).sum(axis=0) / weights.sum())

# 11. Pooling: mean, max, min y concatenación
pool_embs = []
for sent in tokenized:
    vecs = np.array([E[word2idx[t]] for t in sent])
    meanp = vecs.mean(axis=0)
    maxp  = vecs.max(axis=0)
    minp  = vecs.min(axis=0)
    pool_embs.append(np.concatenate([meanp, maxp, minp]))

# 12. Mostrar resultados y dimensiones
print("Dimensiones de vectores:")
print(" Simple avg:", simple_avg[0].shape)
print(" SIF proc :", sif_processed[0].shape)
print(" TF-IDF   :", tfidf_embs[0].shape)
print(" Pool cat :", pool_embs[0].shape)

# 13. Ejemplo de similitud usando promedio simple
from numpy.linalg import norm
def cosine(u, v): return u.dot(v)/(norm(u)*norm(v))
sim = cosine(simple_avg[0], simple_avg[1])
print(f"\nSimilidad coseno entre frase 0 y 1: {sim:.4f}")


#### **Basadas en autoencoder**

Los autoencoders resuelven las debilidades de BoW al aprender codificaciones latentes que capturan orden, sintaxis e interacciones no lineales.

**Arquitectura de encoder-decoder**

Un autoencoder de secuencia implementa:

- **Encoder:** procesa $(x_1,\dots,x_n)$ (embeddings de palabra) mediante una red recurrente:
  $$
    h_t = f(h_{t-1}, x_t),
    \quad
    z = h_n,
  $$
  donde $f$ puede ser LSTM o GRU.

- **Decoder:** genera la secuencia reconstruida:
  $$
    s_t = g(s_{t-1}, z),
    \quad
    \hat x_t = \mathrm{softmax}(W_o s_t),
  $$

La pérdida de reconstrucción se define como entropía cruzada acumulada:

$$
  \mathcal{L}_{AE} = -\sum_{t=1}^n \log P( x_t \mid s_{t-1}, z ).
$$

Esta configuración favorece latentes $z$ que capturan la información esencial de la frase.

**Autoencoder denoising**

Al introducir un operador de ruido $T(\cdot)$ que borra o altera tokens, el encoder recibe $\tilde x = T(x)$ y aprende a reconstruir $x$. La pérdida es:

$$
  \mathcal{L}_{DAE} = -\sum_{t=1}^n \log P( x_t \mid f(\tilde x) ).
$$

Este mecanismo impulsa robustez frente a errores menores y parsinómias de tokenización.

**Autoenconder variacional (VAE)**

El VAE aplica un enfoque probabilístico:

- **Encoder estocástico:** $q_\phi(z\mid x) = \mathcal{N}(\mu(x), \mathrm{diag}(\sigma^2(x)))$.
- **Decoder condicional:** $p_\theta(x\mid z)$.

La función objetivo maximiza la evidencia libre inferior:

$$
  \mathcal{L}_{VAE}
  = \mathbb{E}_{q_\phi} [ \log p_\theta(x\mid z) ]
    - \mathrm{D}_{KL}\bigl(q_\phi(z\mid x) \| p(z)\bigr).
$$

Este esquema favorece latentes continuos y permite generar frases nuevas mediante muestreo en el espacio $z$.

**Vectores Skip‑Thought**

Skip‑Thought extiende la reconstrucción a las oraciones adyacentes:

1. Encoder produce $h_i = f(s_i)$.
2. Dos decoders generan $s_{i-1}$ y $s_{i+1}$.
3. La pérdida combina ambas reconstrucciones:
   $$
     \mathcal{L}_{SkipThought}
     = -\sum_t [ \log P(s_{i-1}[t]\mid h_i) + \log P(s_{i+1}[t]\mid h_i) ].
   $$

Los vectores $h_i$ codifican información semántica inter-oracional.

**Modelos basados en atención (Transformers)**

Aunque no estrictamente autoencoders, los Transformers ofrecen representaciones de frases mediante atención múltiple:

$$
  \mathrm{Attention}(Q,K,V) = \mathrm{softmax}\bigl(\tfrac{QK^\top}{\sqrt{d_k}}\bigr)V.
$$

En BERT, el token especial `[CLS]` produce un vector $h_{CLS}$ que sirve de representación de frase. La autoatención captura dependencias de largo alcance y orden sintáctico.

**Comparativa y aplicaciones prácticas**


| Método            | Captura orden | Robustez OOV | Eficiencia  | Uso típico                            |
|-------------------|---------------|--------------|-------------|---------------------------------------|
| BoW promedio      | No            | Baja         | Alta        | Búsqueda, similitud rápida            |
| BoW ponderado     | No            | Media        | Alta        | Recuperación con stopword handling    |
| Autoencoder Seq2Seq | Sí         | Media        | Media       | Generación de texto, compresión de secuencias |
| VAE               | Sí            | Alta         | Baja        | Generación creativa de frases         |
| Skip‑Thought      | Sí            | Alta         | Baja        | Arquitectura preentrenada general     |
| Transformers      | Sí            | Alta         | Media/Baja  | Tareas de clasificación de frases     |

Las representaciones de frases alimentan distintas aplicaciones:

- **Clasificación de sentimientos:** embeddings de frase como características de entrada a clasificadores lineales o no lineales.
- **Respuesta a preguntas (QA):** mapeo de pregunta y contextos a vectores comparables.
- **Detección de plagio y similitud:** medición de coseno entre vectores de frases largas.
- **Agrupamiento de opiniones:** clustering de reseñas por similitud semántica.



In [ ]:
# Ejemplo detallado de diversos autoencoders para representaciones de frases
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import random

# 1. Preparar vocabulario y datos sintéticos
sentences = [
    ["i", "love", "deep", "learning"],
    ["transformers", "are", "powerful", "models"],
    ["autoencoders", "learn", "latent", "spaces"],
    ["variational", "autoencoder", "is", "probabilistic"],
    ["skip", "thought", "models", "encode", "context"],
]
# Construir vocabulario
vocab = sorted({tok for sent in sentences for tok in sent})
word2idx = {w: i+1 for i, w in enumerate(vocab)}  # 0 para PAD
word2idx["<pad>"] = 0
idx2word = {i:w for w,i in word2idx.items()}
V = len(word2idx)
# Hiperparámetros
emb_dim = 32
hid_dim = 64
batch_size = 2
max_len = max(len(s) for s in sentences)

# Dataset simple que hace padding
class SentenceDataset(Dataset):
    def __init__(self, sents, w2i, max_len):
        self.data = sents
        self.w2i = w2i
        self.max_len = max_len
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        s = self.data[idx]
        seq = [self.w2i[w] for w in s]
        # padding
        seq = seq + [0]*(self.max_len-len(seq))
        return torch.tensor(seq, dtype=torch.long), torch.tensor(seq, dtype=torch.long)

dataset = SentenceDataset(sentences, word2idx, max_len)
loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

# 2. Seq2Seq Autoencoder básico
class Seq2SeqAE(nn.Module):
    def __init__(self, vocab_size, emb_dim, hid_dim):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, emb_dim, padding_idx=0)
        self.encoder = nn.LSTM(emb_dim, hid_dim, batch_first=True)
        self.decoder = nn.LSTM(emb_dim, hid_dim, batch_first=True)
        self.out = nn.Linear(hid_dim, vocab_size)
    def forward(self, x):
        emb = self.embed(x)
        _, (h,n) = self.encoder(emb)
        # Decoder input: usar como primer input same seq (teacher forcing)
        dec_input = emb
        dec_out, _ = self.decoder(dec_input, (h,n))
        logits = self.out(dec_out)
        return logits

# 3. Denoising Autoencoder extiende Seq2SeqAE
class DenoisingAE(Seq2SeqAE):
    def __init__(self, *args, noise_prob=0.3, **kwargs):
        super().__init__(*args, **kwargs)
        self.noise_prob = noise_prob
    def forward(self, x):
        # Corromper tokens con <pad>
        x_noisy = x.clone()
        mask = torch.rand_like(x_noisy.float()) < self.noise_prob
        x_noisy[mask] = 0
        return super().forward(x_noisy)

# 4. VAE Autoencoder
class VAEAE(nn.Module):
    def __init__(self, vocab_size, emb_dim, hid_dim, z_dim=16):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, emb_dim, padding_idx=0)
        self.encoder = nn.LSTM(emb_dim, hid_dim, batch_first=True)
        self.fc_mu = nn.Linear(hid_dim, z_dim)
        self.fc_logvar = nn.Linear(hid_dim, z_dim)
        self.decoder = nn.LSTM(emb_dim+z_dim, hid_dim, batch_first=True)
        self.out = nn.Linear(hid_dim, vocab_size)
    def encode(self, x):
        emb = self.embed(x)
        _, (h,_) = self.encoder(emb)
        h = h.squeeze(0)
        mu = self.fc_mu(h)
        logvar = self.fc_logvar(h)
        return mu, logvar
    def reparam(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std
    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparam(mu, logvar)
        # expand z a secuencia
        seq_z = z.unsqueeze(1).repeat(1, x.size(1), 1)
        emb = self.embed(x)
        dec_in = torch.cat([emb, seq_z], dim=2)
        dec_out, _ = self.decoder(dec_in)
        logits = self.out(dec_out)
        return logits, mu, logvar

# 5. Skip-Thought simplificado: encoder + dos decoders
class SkipThought(nn.Module):
    def __init__(self, vocab_size, emb_dim, hid_dim):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, emb_dim, padding_idx=0)
        self.encoder = nn.GRU(emb_dim, hid_dim, batch_first=True)
        self.decoder_prev = nn.GRU(emb_dim, hid_dim, batch_first=True)
        self.decoder_next = nn.GRU(emb_dim, hid_dim, batch_first=True)
        self.out = nn.Linear(hid_dim, vocab_size)
    def forward(self, x, prev, next_):
        emb = self.embed(x)
        _, h = self.encoder(emb)
        # reconstruir previo
        out_prev, _ = self.decoder_prev(self.embed(prev), h)
        logits_prev = self.out(out_prev)
        # reconstruir siguiente
        out_next, _ = self.decoder_next(self.embed(next_), h)
        logits_next = self.out(out_next)
        return logits_prev, logits_next

# 6. Multi-Head Attention básico
class MultiHeadAttn(nn.Module):
    def __init__(self, hid_dim, n_heads=4):
        super().__init__()
        assert hid_dim % n_heads == 0
        self.d_k = hid_dim // n_heads
        self.n_heads = n_heads
        # Proyecciones
        self.fc_q = nn.Linear(hid_dim, hid_dim)
        self.fc_k = nn.Linear(hid_dim, hid_dim)
        self.fc_v = nn.Linear(hid_dim, hid_dim)
        self.fc_o = nn.Linear(hid_dim, hid_dim)
    def forward(self, q, k, v):
        batch_size = q.size(0)
        # linea a heads
        def proj(x, fc):
            x = fc(x).view(batch_size, -1, self.n_heads, self.d_k)
            return x.permute(0,2,1,3)
        Q = proj(q, self.fc_q)
        K = proj(k, self.fc_k)
        V = proj(v, self.fc_v)
        scores = torch.matmul(Q, K.transpose(-2,-1)) / (self.d_k**0.5)
        attn = torch.softmax(scores, dim=-1)
        x = torch.matmul(attn, V)
        x = x.permute(0,2,1,3).contiguous().view(batch_size, -1, self.n_heads*self.d_k)
        return self.fc_o(x)

# 7. Ejemplo de entrenamiento paso a paso
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
modelos = {
    "AE": Seq2SeqAE(V, emb_dim, hid_dim).to(device),
    "DAE": DenoisingAE(V, emb_dim, hid_dim).to(device),
    "VAE": VAEAE(V, emb_dim, hid_dim).to(device),
}
optims = {name: torch.optim.Adam(m.parameters(), lr=1e-3) for name,m in modelos.items()}
criterion = nn.CrossEntropyLoss(ignore_index=0)

# Un batch para SkipThought (usar sent i=2 y sus vecinos i=1,3)
batch = next(iter(loader))
x, _ = batch
# para SkipThought, crear prev y next padding si es necesario
prev = x.clone()
next_ = x.clone()

for name, modelo in modelos.items():
    modelo.train()
    optims[name].zero_grad()
    outputs = modelo(x.to(device))
    if name=="VAE":
        logits, mu, logvar = outputs
        # perdida reconstruction + KL
        rec = criterion(logits.view(-1,V), x.view(-1).to(device))
        kl = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp()) / x.size(0)
        loss = rec + kl
    else:
        logits = outputs
        loss = criterion(logits.view(-1,V), x.view(-1).to(device))
    loss.backward()
    optims[name].step()
    print(f"{name} loss: {loss.item():.4f}")

# Ejemplo de atención
q = k = v = torch.randn(batch_size, max_len, hid_dim, device=device)
attn = MultiHeadAttn(hid_dim)
out = attn(q,k,v)
print("Forma de salida de  multiHeadAttention :", out.shape)


### **Representaciones distribuidas de oraciones**

Las oraciones, a diferencia de palabras o frases cortas, contienen estructuras sintácticas más complejas y relaciones semánticas de largo alcance, lo que impone nuevos retos a los modelos de representación. Mientras que los embeddings de palabras capturan información léxica y los vectores de frases medianas se centran en composiciones sencillas, las representaciones de oraciones deben integrar:

1. **Información de orden y estructura:** la secuencia de tokens y la dependencia gramatical.
2. **Semántica de alto nivel:** intenciones, relaciones argumentales y tono.
3. **Contexto extraoracional:** en algunos casos, información del párrafo o documento de origen.

Explora dos grandes enfoques: las representaciones generales de oraciones, obtenidas mediante preentrenamiento en tareas de inferencia textual, y las representaciones dirigidas a tareas específicas, donde la arquitectura se adapta y optimiza para el objetivo final.

##### Representación general de oraciones

Los modelos de representación general buscan generar vectores que funcionen bien en una amplia variedad de tareas sin modificación adicional. Dos ejemplos paradigmáticos son **InferSent** y el **Universal Sentence Encoder (USE)**.

**InferSent: Bi‑LSTM con max‑pooling**

**Arquitectura:** InferSent emplea una red bidireccional LSTM (Bi‑LSTM) para procesar la oración:

- Para cada token $w_t$, el encoder genera una representación hacia adelante
  $$
    \overrightarrow{h}_t = \mathrm{LSTM}_f(\overrightarrow{h}_{t-1}, e_{w_t}),
  $$
  y otra hacia atrás
  $$
    \overleftarrow{h}_t = \mathrm{LSTM}_b(\overleftarrow{h}_{t+1}, e_{w_t}).
  $$

- El vector de la oración se construye mediante *max‑pooling* dimensional:
  $$
    v_{\mathrm{sent}}(j) = \max_{1 \le t \le n} \bigl[\overrightarrow{h}_t(j),\overleftarrow{h}_t(j)\bigr]
  $$
  donde $j$ recorre las $d$ dimensiones de los estados ocultos.

**Entrenamiento:** Se entrena en la tarea de **inferencia textual natural (NLI)**, usando datasets como SNLI y MultiNLI. Dado un par de oraciones
- oración base $u$,
- hipótesis $v$,
InferSent crea un vector combinado:
  $$
    x = [\,u,\; v,\; u - v,\; u \odot v\,],
  $$
que se pasa a una red feed‑forward con función de activación ReLU y capa softmax para predecir las clases {entailment, contradiction, neutral}. La función de pérdida es la entropía cruzada:
  $$
    \mathcal{L}_{\text{NLI}} = -\sum_{c \in \{e,n,c\}} y_c \log p_c(x)
  $$
con $y_c$ la etiqueta real y $p_c(x)$ la probabilidad predicha.

**Ventajas:**
- Captura dependencias de largo alcance mediante LSTM.  
- Max‑pooling extrae rasgos sintácticos y semánticos clave.  

**Limitaciones:**
- Entrenamiento costoso en datos NLI.  
- No adapta el embedding para tareas fuera del dominio NLI sin fine‑tuning.

**Universal Sentence Encoder (USE)**

Google desarrolló el USE con dos variantes que equilibran eficiencia y precisión:

1. **Deep Averaging Network (DAN):**  
   - Se calculan embeddings de palabras y bi‑gramas:
     $$ v_i = e_{w_i},\quad g_i = e_{(w_i,w_{i+1})}. $$
   - Se promedian:
     $$ m = \frac{1}{n-1} \sum_{i=1}^{n-1} (v_i + g_i). $$
   - Se proyecta mediante capas densas con ReLU:
     $$ h = \mathrm{ReLU}(W_1 m + b_1),\quad v_{\mathrm{sent}} = \mathrm{ReLU}(W_2 h + b_2). $$
   - El modelo es rápido y de baja latencia, adecuado para aplicaciones en línea.

2. **Transformer Encoder:**  
   - Cada token y bi‑grama recibe un vector de embeddings.  
   - Se aplica *multi-head self-attention* en varias capas:
     $$
       \mathrm{Attention}(Q,K,V) = \mathrm{softmax}\bigl(\tfrac{QK^\top}{\sqrt{d_k}}\bigr)V,
     $$
     donde $Q,K,V$ provienen de proyecciones lineales de las entradas.  
   - Tras $L$ capas, se extrae el vector del token especial [CLS] o se promedian todos los vectores de salida para obtener $v_{\mathrm{sent}}$.

**Preentrenamiento:** USE se entrena con tareas de clasificación de parágrafos y predicción de texto, combinando señales de similitud semántica, minería de parágrafos duplicados y clasificación NLI. La función de pérdida suele ser entropía cruzada sobre múltiples objetivos simultáneos (multi-task), por ejemplo:
  $$
    \mathcal{L}_{\mathrm{USE}} = \lambda_1 \mathcal{L}_{\mathrm{sim}} + \lambda_2 \mathcal{L}_{\mathrm{NLI}} + \lambda_3 \mathcal{L}_{\mathrm{dup}},
  $$
con pesos $\lambda_i$.

**Comparativa:**

| Variante       | Precisión en transferencia | Latencia | Parámetros   |
|----------------|-----------------------------|----------|--------------|
| DAN            | Media                       | Muy baja | ~20M         |
| Transformer    | Alta                        | Media    | ~110M        |

**Aplicaciones generales**

Las representaciones entrenadas en NLI o en tareas multi‑tarea demuestran desempeño sólido en:
- **Clasificación de sentimientos**         
- **Recuperación de información**           
- **Detección de similitud semántica**      
- **Respuesta a preguntas (QA) extractiva**

Además, al ser preentrenamientos, pueden reutilizarse sin modificaciones en tareas con pocos datos propios.

#### **Representación dirigida por tarea**

Cuando el objetivo final difiere significativamente del preentrenamiento general, adaptar la representación de oración al dominio y la tarea específica mejora el rendimiento.

**Atención supervisada sobre tokens relevantes**

Se añade una capa de atención que pondera cada token $h_t$ según su importancia para la tarea:

- Cálculo de puntuaciones:
  $$
    e_t = v_a^\top \tanh(W_h h_t + W_s s),
  $$
  donde $s$ puede ser un vector de estado inicial o contexto externo, y $v_a,W_h,W_s$ son parámetros aprendidos.

- Normalización con softmax:
  $$
    \alpha_t = \frac{\exp(e_t)}{\sum_{k=1}^n \exp(e_k)}.
  $$

- Vector de oración ponderado:
  $$
    v_{\mathrm{task}} = \sum_{t=1}^n \alpha_t h_t.
  $$

Este mecanismo permite al modelo enfocarse en fragmentos críticos (por ejemplo, negaciones o entidades) y reduce el ruido de tokens irrelevantes.

**Multi-task Learning y pérdida conjunta**

Al entrenar simultáneamente en la tarea principal y tareas auxiliares (por ejemplo, etiquetado de entidades, detección de sentimientos), se define una pérdida combinada:
  $$
    \mathcal{L} = \lambda_p \mathcal{L}_{\mathrm{principal}} + \sum_{i=1}^m \lambda_i \mathcal{L}_{\mathrm{aux}_i},
  $$
con $\lambda_p,\lambda_i$ pesos de cada tarea y pérdidas específicas $\mathcal{L}_{\mathrm{aux}}$ (entropía cruzada, CRF, etc.). Esta configuración refuerza las características compartidas y mejora la generalización.

**Fine‑tuning de modelos preentrenados (Transfer Learning)**

Métodos modernos como BERT incorporan un preentrenamiento masivo y capas de Transformer. Para tareas de clasificación de oraciones se procede a:

1. **Añadir cabecera de clasificación:** un vector de salida `[CLS]` se conecta a una capa densa:
   $$
     \hat y = \mathrm{softmax}(W_{cls} h_{[CLS]} + b_{cls}).
   $$
2. **Fine‑tuning end‑to‑end:** todos los parámetros de BERT y la capa adicional se actualizan mínimo un par de epochs con tasa de aprendizaje baja.
3. **Pérdida de entropía cruzada:**
   $$
     \mathcal{L}_{\mathrm{BERT-tune}} = -\sum_{c} y_c \log \hat y_c.
   $$

Este enfoque adapta las representaciones de oraciones a características propias de la tarea (voz activa/pasiva, tono, polaridad).

**Atención multi-head y pooling adaptativo**

Al utilizar Transformers, se pueden extraer representaciones dirigidas mediante *attention pooling*:

1. **Multi-head Attention Output:** de la última capa se obtienen vectores $H = [h_1,\dots,h_n]$.
2. **Cálculo de logits de atención para cada cabecera:**
   $$
     A = \mathrm{softmax}\bigl(W_A H^\top\bigr),
   $$
   produciendo pesos $\alpha_{t}^{(i)}$ por head.
3. **Pooling adaptativo:**
   $$
     v_{\mathrm{mh}} = \bigl[\sum_t \alpha_t^{(1)} h_t;\dots;\sum_t \alpha_t^{(k)} h_t\bigr],
   $$
   concatenando los outputs de $k$ cabeceras para capturar perspectivas múltiples.

**Comparativa de enfoques dirigidos**

| Método                    | Adaptación | Dependencia de datos | Costo computacional |
|---------------------------|------------|----------------------|---------------------|
| Capa atención simple      | Media      | Moderada             | Baja–Media          |
| Multi-task Learning       | Alta       | Alta                 | Media–Alta          |
| Fine‑tuning BERT          | Muy alta   | Moderada–Alta        | Alta                |
| Attention pooling adapt.  | Media      | Moderada             | Media               |

Cada técnica presenta un balance entre la cantidad de datos etiquetados necesarios, la flexibilidad para tareas nuevas y el coste de entrenamiento.


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import BertModel, BertTokenizer

# Datos de ejemplo: batch de oraciones
sentences = [
    "Deep learning models capture complex semantics.",
    "Transformers excel at long-range dependencies.",
    "Sentence representations power many NLP tasks."
]
batch_size = len(sentences)
max_len = 10  # Número máximo de tokens por oración (relleno)

# 1. Tokenización simple (solo para ilustrar, se recomienda usar un tokenizer real) 
# Se crea un vocabulario con índice para cada palabra
vocab = {w: i+1 for i, w in enumerate(set(" ".join(sentences).split()))}
vocab["<pad>"] = 0  # Índice para padding

# Función para tokenizar y aplicar padding a las oraciones
def tokenize_and_pad(sents, vocab, max_len):
    tokenized = []
    for s in sents:
        tokens = [vocab.get(w, 0) for w in s.split()][:max_len]
        tokens = tokens + [0] * (max_len - len(tokens))  # Padding
        tokenized.append(tokens)
    return torch.tensor(tokenized, dtype=torch.long)

inputs = tokenize_and_pad(sentences, vocab, max_len)

# 2. Modelo InferSent: Bi-LSTM + max pooling
class InferSent(nn.Module):
    def __init__(self, vocab_size, emb_dim, hid_dim):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, emb_dim, padding_idx=0)  # Capa de embedding
        self.encoder = nn.LSTM(emb_dim, hid_dim, bidirectional=True, batch_first=True)  # Bi-LSTM

    def forward(self, x):
        emb = self.emb(x)  # (Batch, Longitud, Dimensión)
        h, _ = self.encoder(emb)  # (Batch, Longitud, 2 * Hidden)
        pooled, _ = torch.max(h, dim=1)  # Max pooling a lo largo de la secuencia
        return pooled  # Representación final (Batch, 2H)

#3. Modelo USE basado en Deep Averaging Network (DAN) 
class USE_DAN(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim, out_dim):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, emb_dim, padding_idx=0)
        self.fc1 = nn.Linear(emb_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, out_dim)

    def forward(self, x):
        emb = self.emb(x).mean(dim=1)  # Promedio de embeddings por oración
        h = F.relu(self.fc1(emb))     # Primera capa oculta con ReLU
        out = F.relu(self.fc2(h))     # Capa de salida con ReLU
        return out  # Representación final

# 4. Modelo USE con codificador Transformer (una sola capa)
class USE_Trans(nn.Module):
    def __init__(self, vocab_size, emb_dim, n_heads, hidden_dim, n_layers):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, emb_dim, padding_idx=0)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=emb_dim, nhead=n_heads, dim_feedforward=hidden_dim
        )
        self.encoder = nn.TransformerEncoder(encoder_layer, num_layers=n_layers)

    def forward(self, x):
        emb = self.emb(x).permute(1,0,2)  # Cambia a formato (Longitud, Batch, Dim)
        out = self.encoder(emb)          # Salida del Transformer
        cls = out[0]                     # Usar primer token como representación tipo [CLS]
        return cls

#  5. Mecanismo de atención para hacer pooling sobre salidas de Bi-LSTM 
class AttentionPool(nn.Module):
    def __init__(self, hid_dim):
        super().__init__()
        self.attn = nn.Linear(hid_dim, 1)  # Capa de atención

    def forward(self, h):
        # h tiene forma (Batch, Longitud, Dim)
        scores = self.attn(h).squeeze(-1)        # Obtiene puntuaciones (Batch, Longitud)
        weights = F.softmax(scores, dim=1)       # Normaliza puntuaciones
        pooled = torch.bmm(weights.unsqueeze(1), h).squeeze(1)  # Aplica atención
        return pooled  # Representación con atención

# 6. Clasificador con fine-tuning de BERT
class BERTClassifier(nn.Module):
    def __init__(self, out_dim):
        super().__init__()
        self.bert = BertModel.from_pretrained("bert-base-uncased")  # Cargar BERT preentrenado
        self.cls_fc = nn.Linear(self.bert.config.hidden_size, out_dim)  # Capa final

    def forward(self, input_ids, attention_mask=None):
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        cls_token = outputs.last_hidden_state[:, 0, :]  # Representación del token [CLS]
        logits = self.cls_fc(cls_token)  # Logits de salida
        return logits

# 7. Modelo multi-tarea con InferSent compartido + dos salidas
class MultiTaskSent(nn.Module):
    def __init__(self, encoder, out1, out2):
        super().__init__()
        self.encoder = encoder
        self.head1 = nn.Linear(encoder.encoder.hidden_size*2, out1)  # Tarea 1
        self.head2 = nn.Linear(encoder.encoder.hidden_size*2, out2)  # Tarea 2

    def forward(self, x):
        rep = self.encoder(x)  # Representación de oración (Batch, 2H)
        y1 = self.head1(rep)   # Salida para tarea 1
        y2 = self.head2(rep)   # Salida para tarea 2
        return y1, y2

# Instanciar modelos con parámetros de ejemplo 
vocab_size = len(vocab)
emb_dim = 64
hid_dim = 128

use_dan = USE_DAN(vocab_size, emb_dim, 64, 128)
infer_sent = InferSent(vocab_size, emb_dim, hid_dim)
use_trans = USE_Trans(vocab_size, emb_dim, n_heads=4, hidden_dim=256, n_layers=1)
attn_pool = AttentionPool(hid_dim*2)
multi_task = MultiTaskSent(infer_sent, out1=3, out2=2)

# Pruebas de paso hacia adelante (forward)
x = inputs  # Tensor de entrada (Batch, Longitud)

print(" Forma de  InferSent:", infer_sent(x).shape)
print("Forma del USE DAN:", use_dan(x).shape)
print("Forma de USE Trans:", use_trans(x).shape)

# Obtener salidas del Bi-LSTM para atención
h_bilstm, _ = infer_sent.encoder(use_dan.emb(x))
print("Forma de salida del AttentionPool:", attn_pool(h_bilstm).shape)

# Salidas del modelo multitarea
mt1, mt2 = multi_task(x)
print("Forma de las cabeceras de  multiTask:", mt1.shape, mt2.shape)

# Clasificación con BERT
# Tokenización con tokenizer real
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
encoded = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")

# Instanciar clasificador BERT y hacer forward
bert_clf = BERTClassifier(out_dim=2)
logits = bert_clf(encoded["input_ids"], attention_mask=encoded["attention_mask"])
print("Forma de los logits del BERTClassifier:", logits.shape)



### **Representaciones distribuidas de documentos** 

Doc2vec nos permite aprender directamente las representaciones de textos de longitud arbitraria (frases, oraciones, párrafos y documentos), teniendo en cuenta el contexto de las palabras del texto.

Esto es similar a Word2vec en términos de su arquitectura general, excepto que, además de los vectores de palabras, también aprende un "vector de párrafo" que aprende una representación del texto completo (es decir, con palabras en contexto). Cuando se aprende con un corpus grande de muchos textos, los vectores de párrafo son únicos para un texto determinado (donde "texto" puede significar cualquier fragmento de texto de longitud arbitraria), mientras que los vectores de palabras se compartirán en todos los textos.  

Hay dos arquitecturas del modelo Doc2Vec, que es una extensión de Word2Vec diseñada para generar representaciones vectoriales no solo para palabras sino también para piezas de texto más grandes como oraciones, párrafos y documentos. Estas representaciones vectoriales son útiles para muchas tareas de procesamiento del lenguaje natural, como la clasificación de textos y la búsqueda semántica. Aquí están las dos arquitecturas: 

**Memoria distribuida (DM)**: 

En el modelo DM de Doc2Vec, cada palabra y el párrafo (o documento) entero tienen su propio vector de aprendizaje único en una "Paragraph Matrix" y en una "Word Matrix", respectivamente. 

Durante el entrenamiento, el modelo intenta predecir la siguiente palabra en un contexto dada una ventana de palabras y el vector único del párrafo/documento. 

Los vectores de las palabras y del párrafo se pueden promediar o concatenar antes de enviarlos a una capa de clasificador, que intenta predecir la palabra siguiente. 

El objetivo es que al final del entrenamiento, el vector del párrafo capture la esencia del texto, lo que hace posible usar este vector para tareas de clasificación o comparación de similitud. 

**Bolsa de palabras distribuidas (DBOW)**: 

El modelo DBOW funciona de manera inversa al DM. Ignora el contexto de las palabras y, en su lugar, fuerza al modelo a predecir las palabras en un párrafo/documento dada solo la identificación del párrafo (es decir, su vector único). 

No hay una capa de promedio o concatenación; el modelo directamente predice las palabras a partir del vector del párrafo. 

Al igual que en el modelo DM, el vector del párrafo se entrena para representar el contenido completo del párrafo/documento. 

DBOW es eficaz para grandes conjuntos de datos donde la semántica puede ser capturada incluso sin el orden exacto de las palabras. 

Ambos métodos son útiles para aprender representaciones vectoriales que reflejan el significado de los párrafos o documentos, aunque capturan diferentes aspectos de los datos: DM toma en cuenta el orden de las palabras, mientras que DBOW se centra en la ocurrencia de las palabras. Estos vectores resultantes pueden ser utilizados en diversas tareas, tales como agrupación de documentos, clasificación y búsqueda por similitud semántica. 

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import random

# 1. Datos sintéticos: párrafos y frases
paragraphs = {
    0: ["the cat sat on the mat".split(), "the cat is happy".split()],
    1: ["dogs are loyal animals".split(), "my dog loves to play".split()],
    2: ["deep learning models require lots of data".split()]
}

# 2. Construir vocabulario
all_words = set(w for para in paragraphs.values() for sent in para for w in sent)
word2idx = {w: i for i, w in enumerate(sorted(all_words), start=1)}
word2idx["<pad>"] = 0
idx2word = {i: w for w, i in word2idx.items()}
V = len(word2idx)
P = len(paragraphs)

# 3. Parámetros de modelo
emb_dim = 50
context_size = 2  # ventana izquierda y derecha
batch_size = 4

# 4. Dataset DM: (para cada palabra central, contexto + para_id) -> target word
class Doc2VecDMDataset(Dataset):
    def __init__(self, paragraphs, word2idx, context_size):
        self.samples = []
        for pid, sents in paragraphs.items():
            for sent in sents:
                indices = [word2idx[w] for w in sent]
                for i, target in enumerate(indices):
                    context = []
                    for offset in range(-context_size, context_size+1):
                        j = i + offset
                        if offset != 0 and 0 <= j < len(indices):
                            context.append(indices[j])
                    if context:
                        self.samples.append((pid, context, target))
    def __len__(self):
        return len(self.samples)
    def __getitem__(self, idx):
        pid, context, target = self.samples[idx]
        # pad context to fixed size
        pad = [0] * (2*context_size - len(context))
        context = context + pad
        return torch.tensor(pid), torch.tensor(context), torch.tensor(target)

# 5. Modelo DM
class Doc2VecDM(nn.Module):
    def __init__(self, vocab_size, para_count, emb_dim, context_size):
        super().__init__()
        self.word_emb = nn.Embedding(vocab_size, emb_dim, padding_idx=0)
        self.para_emb = nn.Embedding(para_count, emb_dim)
        self.linear = nn.Linear((2*context_size)*emb_dim + emb_dim, vocab_size)
    def forward(self, pid, context_idxs):
        # context_idxs: (B, 2*context_size)
        w_ctx = self.word_emb(context_idxs)  # (B, 2C, D)
        w_ctx = w_ctx.view(w_ctx.size(0), -1)  # (B, 2C*D)
        p_vec = self.para_emb(pid)             # (B, D)
        x = torch.cat([w_ctx, p_vec], dim=1)   # (B, 2C*D + D)
        logits = self.linear(x)                # (B, V)
        return logits

# 6. Modelo DBOW
class Doc2VecDBOW(nn.Module):
    def __init__(self, para_count, vocab_size, emb_dim):
        super().__init__()
        self.para_emb = nn.Embedding(para_count, emb_dim)
        self.linear = nn.Linear(emb_dim, vocab_size)
    def forward(self, pid):
        p_vec = self.para_emb(pid)  # (B, D)
        logits = self.linear(p_vec) # (B, V)
        return logits

# 7. Preparar dataloaders
dm_dataset = Doc2VecDMDataset(paragraphs, word2idx, context_size)
dm_loader = DataLoader(dm_dataset, batch_size=batch_size, shuffle=True)

# Para DBOW, generamos muestras (para_id -> random word from paragraph)
dbow_samples = []
for pid, sents in paragraphs.items():
    words = [word2idx[w] for sent in sents for w in sent]
    for w in words:
        dbow_samples.append((pid, w))
class DBOWDataset(Dataset):
    def __init__(self, samples):
        self.samples = samples
    def __len__(self): return len(self.samples)
    def __getitem__(self, i): return torch.tensor(self.samples[i][0]), torch.tensor(self.samples[i][1])
dbow_loader = DataLoader(DBOWDataset(dbow_samples), batch_size=batch_size, shuffle=True)

# 8. Entrenamiento de ejemplo
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
dm_model = Doc2VecDM(V, P, emb_dim, context_size).to(device)
dbow_model = Doc2VecDBOW(P, V, emb_dim).to(device)
criterion = nn.CrossEntropyLoss()
dm_opt = optim.Adam(dm_model.parameters(), lr=0.01)
dbow_opt = optim.Adam(dbow_model.parameters(), lr=0.01)

# 9. Un paso de entrenamiento DM
pid, ctx, tgt = next(iter(dm_loader))
pid, ctx, tgt = pid.to(device), ctx.to(device), tgt.to(device)
dm_model.train()
dm_opt.zero_grad()
logits = dm_model(pid, ctx)
loss_dm = criterion(logits, tgt)
loss_dm.backward()
dm_opt.step()
print(f"Perdida DM: {loss_dm.item():.4f}")

# 10. Un paso de entrenamiento DBOW
pid2, tgt2 = next(iter(dbow_loader))
pid2, tgt2 = pid2.to(device), tgt2.to(device)
dbow_model.train()
dbow_opt.zero_grad()
logits2 = dbow_model(pid2)
loss_dbow = criterion(logits2, tgt2)
loss_dbow.backward()
dbow_opt.step()
print(f"Perdida DBOW : {loss_dbow.item():.4f}")

# 11. Obtener embeddings de párrafo y palabras
with torch.no_grad():
    para_vecs = dm_model.para_emb.weight  # (P, D)
    word_vecs = dm_model.word_emb.weight  # (V, D)
print("Para[0] muestra de embedding :", para_vecs[0][:5])


### **Representación orientada a tareas**

En entornos de procesamiento de lenguaje natural (NLP) y recuperación de información (IR), los documentos adoptan variadas formas y tamaños, desde párrafos breves hasta informes completos. Si bien los embeddings generales de documentos (como Doc2Vec en sus variantes DM y DBOW) ofrecen vectores eficaces para tareas amplias, en escenarios específicos es fundamental **afinar** estas representaciones para reflejar las necesidades particulares de la aplicación final.

La **representación orientada a tareas** surge para reducir la brecha entre la generalidad del embedding base y la especialización requerida por tareas como resumen automático, clasificación de textos o sistemas de pregunta‑respuesta (QA).

Exploramos dos estrategias principales: el **fine‑tuning** de embeddings preentrenados mediante retropropagación y el **aprendizaje multi‑tarea** ("multi-task learning"), donde múltiples objetivos se entrenan de forma conjunta. Ambas metodologías se apoyan en formulaciones matemáticas rigurosas para adaptar vectores de documento a dominios concretos.

#### **Fine‑tuning de embeddings de documento**

El proceso de fine‑tuning consiste en tomar un modelo preentrenado, por ejemplo, un encoder de documentos en arquitectura Transformer o un Doc2Vec y continuar su entrenamiento en un conjunto etiquetado para la tarea meta. Imagina que disponemos de un conjunto de documentos $\{D_i\}_{i=1}^N$, cada uno con su embedding inicial $v_i^{(0)}\in \mathbb{R}^d$, generado por la red base. Para una tarea de clasificación, asociamos a cada documento una etiqueta $y_i\in \{1,\dots,C\}$.

**Función objetivo y retropropagación**

Se define una capa de clasificación sobre el embedding:

$$
  \hat p_i = \mathrm{softmax}(W_{c} v_i + b_{c}),
$$

donde $W_{c}\in \mathbb{R}^{C\times d}$ y $b_{c}\in\mathbb{R}^C$ son parámetros de la capa de salida, y

$$
  \hat p_{i,j} = \frac{\exp\bigl((W_{c} v_i + b_{c})_j\bigr)}{\sum_{k=1}^C \exp\bigl((W_{c} v_i + b_{c})_k\bigr)}.
$$

La pérdida de entropía cruzada para un solo ejemplo es:

$$
  \mathcal{L}_i = -\sum_{j=1}^C \mathbb{I}_{[y_i=j]} \log\bigl(\hat p_{i,j}\bigr).
$$

El fine‑tuning también actualiza los parámetros internos $\theta$ del encoder de documento (sea Transformer, RNN o Doc2Vec). El objetivo total sobre el dataset es:

$$
  \mathcal{L}_{\mathrm{fine}} = \frac{1}{N} \sum_{i=1}^N \mathcal{L}_i + \lambda\,\lVert \theta \rVert^2,
$$

donde $\lambda$ aplica regularización L2 para evitar sobreajuste. El descenso por gradiente (o variantes como Adam) actualiza simultáneamente $W_c, b_c$ y $\theta$:

$$
  \theta \leftarrow \theta - \eta \frac{\partial \mathcal{L}_{\mathrm{fine}}}{\partial \theta},
  \quad
  W_{c} \leftarrow W_{c} - \eta \frac{\partial \mathcal{L}_{\mathrm{fine}}}{\partial W_{c}},
$$

y $\eta$ es la tasa de aprendizaje. Este ajuste continuo permite que el embedding capture patrones cruciales para la clasificación de documentos, tales como la presencia de términos discriminativos, la estructura argumental y la coherencia temática.

**Ejemplo: fine‑tuning para resumen extractivo**

En resumen automático extractivo, se entrena un modelo que aprueba o rechaza cada frase $s_{i,j}$ de un documento $D_i$. Se obtiene el embedding de frase $v_{i,j}$ y se aplica:

$$
  \hat p_{i,j} = \sigma\bigl(w_{s}^{\top} v_{i,j} + b_{s}\bigr),
$$

con $\sigma(x) = 1/(1+e^{-x})$. La pérdida binaria log‑loss:

$$
  \mathcal{L}_{i,j} = -\bigl[y_{i,j}\log\hat p_{i,j} + (1-y_{i,j})\log(1-\hat p_{i,j})\bigr].
$$

Fine‑tuning ajusta tanto $w_s, b_s$ como $\theta$ del encoder de frase, permitiendo que la representación enfatice contenidos relevantes para el resumen.

**Ventajas y desafíos del fine‑tuning**

- **Ventajas:**
  - Alta precisión en el dominio específico.  
  - Aprovechamiento de conocimiento general preentrenado.  

- **Desafíos:**
  - Riesgo de sobreajuste con pocos datos etiquetados.  
  - Coste computacional elevado si el encoder es muy grande.  
  - Necesidad de ajustes cuidadosos de la tasa de aprendizaje $\eta$ y regularización $\lambda$.  

#### **Aprendizaje multi‑tarea (Multi‑Task Learning)**

El **multi-task learning** optimiza simultáneamente varias tareas relacionadas, compartiendo representaciones subyacentes y promoviendo la transferencia de conocimiento entre tareas. Para documentos, esto puede incluir tareas como clasificación temática, análisis de sentimiento y extracción de entidades.

**Formulación matemática**

Supongamos $T$ tareas, cada una con función de pérdida $\mathcal{L}^{(t)}$. Un ejemplo de tareas podría ser:

1. **Resumen extractivo**: $\mathcal{L}^{(1)}$.  
2. **Clasificación de tema**: $\mathcal{L}^{(2)}$.  
3. **Detección de sentimiento**: $\mathcal{L}^{(3)}$.

Se comparte el encoder de documento con parámetros $\theta_s$ (“shared”), y cada tarea posee parámetros específicos $\theta_t$ ("task‑specific"). La pérdida total se define como:

$$
  \mathcal{L}_{\mathrm{MTL}} = \sum_{t=1}^T \lambda_t \frac{1}{N_t} \sum_{i=1}^{N_t} \mathcal{L}_i^{(t)}(\theta_s,\theta_t),
$$

con pesos $\lambda_t>0$ que equilibran la importancia de cada tarea y $N_t$ el número de ejemplos de la tarea $t$. Las actualizaciones por gradiente son:

$$
  \theta_s \leftarrow \theta_s - \eta \sum_{t=1}^T \lambda_t \frac{\partial \mathcal{L}^{(t)}}{\partial \theta_s},
$$
$$
  \theta_t \leftarrow \theta_t - \eta \lambda_t \frac{\partial \mathcal{L}^{(t)}}{\partial \theta_t},\quad t=1,\dots,T.
$$

Este esquema aprovecha la señal adicional de tareas relacionadas para mejorar la calidad del embedding compartido $\theta_s$, reduciendo la probabilidad de sobreajuste y mejorando la generalización.

**Ejemplo: tareas combinadas de QA y clasificación**

En un sistema de pregunta‑respuesta, podemos combinar:
- **Respuesta extractiva (QA):** $\mathcal{L}^{(1)}$ entropía cruzada sobre spans de texto.  
- **Clasificación de tema:** $\mathcal{L}^{(2)}$ entropía cruzada sobre categorías.

La capa compartida $\theta_s$ aprende características sintácticas útiles para identificar tanto las secciones relevantes de texto como el dominio temático, mientras que $\theta_1$ y $\theta_2$ se especializan en cada tarea.

**Regularización implícita y explícita**

- **Regularización implícita:** el multi-task actúa como un "armado inductivo", ya que la necesidad de resolver varias tareas impide sobreajustar una sola.  
- **Regularización explícita:** se pueden añadir términos penalizadores similares a L2 o dropout en $\theta_s$ para mejorar la robustez.

**Weighting dinámico de tareas**

Determinar pesos $\lambda_t$ adecuados es crítico. Estrategias dinámicas incluyen:
1. **Uncertainty weighting** (Kendall et al., 2018): el peso se ajusta según la varianza de la pérdida de cada tarea, definiendo $\lambda_t = 1/(2\sigma_t^2)$ y aprendiendo $\sigma_t$.  
2. **GradNorm** (Chen et al., 2018): equilibra magnitudes de gradiente para que ninguna tarea domine la actualización de $\theta_s$.  
Ambos métodos conducen a mejoras en la convergencia y a representaciones más equilibradas.

**Comparativa y consideraciones de diseño**

| Aspecto                | Fine‑tuning simple  | Multi‑Task Learning      | Ventaja clave                               |
|------------------------|---------------------|--------------------------|---------------------------------------------|
| Dependencia de datos   | Alta                | Moderada                 | MTL necesita múltiples anotaciones          |
| Costo computacional    | Alto (por tarea)    | Compartido                | MTL entrena un solo encoder para varias     |
| Riesgo de sobreajuste  | Alto (pocos datos)  | Bajo                     | MTL regulariza implícitamente               |
| Adaptabilidad          | Alta                | Media                    | Fine‑tuning específico para cada tarea      |


**Extensiones y tendencias actuales**

1. **Meta‑Learning para NLP:** se entrena un modelo que aprende a adaptarse rápidamente a tareas nuevas con pocos ejemplos (learning-to-learn).  
2. **Adapters en Transformers:** módulos ligeros insertados entre capas preentrenadas, entrenables por tarea, reducen drásticamente el número de parámetros ajustados.  
3. **Prompt‑Tuning:** en modelos de lenguaje grande, se optimizan vectores de "prompt" para dirigir el encoder a la tarea deseada sin modificar parámetros básicos.


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

# 1. Datos sintéticos: documentos y etiquetas para dos tareas
docs = [
    "deep learning improves nlp tasks",
    "distributed representations capture semantics",
    "fine tuning adapts embeddings to domain",
    "multi task learning shares knowledge",
    "transformers use attention mechanisms"
]
labels_task1 = [0, 1, 0, 1, 1]      # e.g., topic classification (2 clases)
labels_task2 = [1, 0, 1, 0, 1]      # e.g., sentiment (positivo=1/negativo=0)

# 2. Tokenización simple y vocabulario
tokenized = [doc.split() for doc in docs]
vocab = set(word for sent in tokenized for word in sent)
word2idx = {w: i+1 for i, w in enumerate(sorted(vocab))}
word2idx["<pad>"] = 0
vocab_size = len(word2idx)

# 3. Dataset y DataLoader
max_len = max(len(sent) for sent in tokenized)
class DocDataset(Dataset):
    def __init__(self, tokenized, labels1, labels2, w2i, max_len):
        self.data = tokenized
        self.labels1 = labels1
        self.labels2 = labels2
        self.w2i = w2i
        self.max_len = max_len
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        tokens = self.data[idx]
        ids = [self.w2i[w] for w in tokens]
        # padding
        ids = ids + [0] * (self.max_len - len(ids))
        return torch.tensor(ids), torch.tensor(self.labels1[idx]), torch.tensor(self.labels2[idx])

dataset = DocDataset(tokenized, labels_task1, labels_task2, word2idx, max_len)
loader = DataLoader(dataset, batch_size=2, shuffle=True)

# 4. Encoder de documento: promedio de embeddings seguido de MLP
class DocEncoder(nn.Module):
    def __init__(self, vocab_size, emb_dim, hidden_dim):
        super().__init__()
        self.emb = nn.Embedding(vocab_size, emb_dim, padding_idx=0)
        self.fc = nn.Linear(emb_dim, hidden_dim)
    def forward(self, x):
        # x: (B, L)
        emb = self.emb(x)                # (B, L, D)
        mean = emb.mean(dim=1)           # (B, D)
        h = torch.relu(self.fc(mean))    # (B, H)
        return h

# 5. Fine-tuning para tarea de clasificación (task1)
class FineTuneClassifier(nn.Module):
    def __init__(self, encoder, hidden_dim, num_classes):
        super().__init__()
        self.encoder = encoder
        self.classifier = nn.Linear(hidden_dim, num_classes)
    def forward(self, x):
        v = self.encoder(x)              # embedding víctorial v_i
        logits = self.classifier(v)      # W_c v_i + b_c
        return logits, v

# 6. Entrenamiento de fine-tuning
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
enc = DocEncoder(vocab_size, emb_dim=16, hidden_dim=32).to(device)
model_ft = FineTuneClassifier(enc, hidden_dim=32, num_classes=2).to(device)
opt_ft = optim.Adam(model_ft.parameters(), lr=1e-3, weight_decay=1e-4)  # lambda L2
criterion = nn.CrossEntropyLoss()

# Mostrar fórmula de pérdida:
# L_fine = 1/N sum_i [-sum_j I[y_i=j] log p_{i,j}] + lambda ||theta||^2

for epoch in range(3):
    model_ft.train()
    total_loss = 0
    for x, y1, _ in loader:
        x, y1 = x.to(device), y1.to(device)
        logits, v = model_ft(x)
        loss = criterion(logits, y1)
        opt_ft.zero_grad()
        loss.backward()
        opt_ft.step()
        total_loss += loss.item()
    print(f"[Fine-tuning] Epoch {epoch+1}, Loss: {total_loss/len(loader):.4f}")

# 7. Multi-task Learning: compartir encoder, dos cabeceras
class MultiTaskModel(nn.Module):
    def __init__(self, encoder, hidden_dim, num_classes1, num_classes2):
        super().__init__()
        self.encoder = encoder
        self.head1 = nn.Linear(hidden_dim, num_classes1)  # theta_1
        self.head2 = nn.Linear(hidden_dim, num_classes2)  # theta_2
    def forward(self, x):
        shared = self.encoder(x)        # theta_s
        out1 = self.head1(shared)       # logits tarea1
        out2 = self.head2(shared)       # logits tarea2
        return out1, out2

# 8. Entrenamiento multi-task
mtl_model = MultiTaskModel(enc, hidden_dim=32, num_classes1=2, num_classes2=2).to(device)
opt_mtl = optim.Adam(mtl_model.parameters(), lr=1e-3)
# Pesos lambda_t
lambda1, lambda2 = 0.7, 0.3

# Pérdida total: L_MTL = lambda1 * L1 + lambda2 * L2
for epoch in range(3):
    mtl_model.train()
    total_mtl = 0
    for x, y1, y2 in loader:
        x, y1, y2 = x.to(device), y1.to(device), y2.to(device)
        logits1, logits2 = mtl_model(x)
        loss1 = criterion(logits1, y1)
        loss2 = criterion(logits2, y2)
        loss = lambda1 * loss1 + lambda2 * loss2
        opt_mtl.zero_grad()
        loss.backward()
        opt_mtl.step()
        total_mtl += loss.item()
    print(f"[Multi-Task] Epoca {epoch+1}, Loss: {total_mtl/len(loader):.4f}")

# 9. Evaluación rápida
mtl_model.eval()
with torch.no_grad():
    x, y1, y2 = next(iter(loader))
    x = x.to(device)
    p1, p2 = mtl_model(x)
    pred1 = p1.argmax(dim=1)
    pred2 = p2.argmax(dim=1)
    print("Pred task1:", pred1.cpu().tolist(), " True:", y1.tolist())
    print("Pred task2:", pred2.cpu().tolist(), " True:", y2.tolist())


### Ejercicios

#### 1. Embeddings de palabras avanzados

**Objetivo**: Comparar calidad y sesgos de distintos embeddings pre‑entrenados (Word2Vec, GloVe, FastText).

- **Tarea 1.1**: Elige tres modelos pre‑entrenados (por ejemplo, Google News Word2Vec, GloVe Common Crawl y FastText Wikipedia).  
  - Diseña un benchmark de analogías (p. ej. "king – man + woman ≈ queen") y cálculo de similitud de pares (p. ej. datasets of RG65 o WordSim-353).  
  - Mide exactitud en analogías y correlación Spearman en similitud.  
  - Analiza la varianza de desempeño con distintas dimensiones (50, 100, 300).

- **Tarea 1.2**: Evalúa sesgos de género y raciales en cada embedding.  
  - Usa el método de "Word embedding association test" (WEAT) para cuantificar sesgo.  
  - Propón una estrategia de mitigación (p. ej. neutralización y equalización).  
  - Repite WEAT tras mitigación y comenta resultados.


#### 2. Representaciones de frases con BoW

**Objetivo**: Implementar y contrastar los esquemas BoW: promedio simple, SIF y TF‑IDF ponderado.

- **Tarea 2.1**: Dado un corpus de frases pareadas (p. ej. STS Benchmark en inglés),  
  - Calcula para cada frase sus representaciones con (i) promedio simple, (ii) SIF ($a=10^{-3}$) y (iii) TF‑IDF.  
  - Mide correlación coseno vs. anotaciones humanas.  
  - Grafica con TSNE las frases de mayor y menor similitud según cada método.

- **Tarea 2.2**: Incorpora pooling adicional: max‑pooling, min‑pooling y concatenación mean | max | min.  
  - Evalúa cómo cambia la correlación STS y el coste de cómputo.  
  - Concluye qué esquema es más adecuado en un entorno de baja latencia.


#### 3. Autoencoders de frases

**Objetivo**: Profundizar en codificaciones latentes que capturen orden y robustez.

- **Tarea 3.1**: Entrena un **Seq2Seq AE** para reconstruir frases de un corpus (p. ej. IMDB).  
  - Mide pérdida de reconstrucción y porcentaje de tokens correctamente reconstruidos.  
  - Verifica que el vector latente $z$ captura información de orden (intercambia dos palabras y mide la pérdida).

- **Tarea 3.2**: Extiende a **Denoising AE**:  
  - Aplica ruido (borra o sustituye tokens aleatorios) y entrena a reconstruir la frase original.  
  - Evalúa robustez ante ruido: compara calidad de reconstrucción vs. AE estándar.

- **Tarea 3.3**: Implementa un **Variational AE (VAE)** de frases.  
  - Visualiza el espacio latente usando interpolaciones lineales entre dos frases y genera oraciones intermedias.  
  - Comenta ventajas e inconvenientes de la entropía KL en el entrenamiento (p. ej. "posterior collapse").


#### 4. Skip‑Thought y Transformers

**Objetivo**: Explorar representaciones inter‑oracionales y basadas en atención.

- **Tarea 4.1**: Entrena un Skip‑Thought (encoder Bi‑GRU + dos decoders) sobre un corpus de párrafos (p. ej. BookCorpus).  
  - Evalúa las representaciones en tareas de **NLI** (SNLI) con un clasificador ad hoc.  
  - Contrástalas con un modelo InferSent entrenado desde cero.

- **Tarea 4.2**: Implementa un encoder Transformer (una capa) y extrae el token `[CLS]` como vector de oración.  
  - Fine‑tunea este encoder en SNLI y mide exactitud vs. InferSent y Skip‑Thought.  
  - Analiza la importancia del número de heads y la profundidad (1 vs. 2 capas).


#### 5. Doc2Vec: DM vs. DBOW

**Objetivo**: Aprender y comparar representaciones de documentos.

- **Tarea 5.1**: Entrena Doc2Vec DM y DBOW sobre un corpus de noticias (p. ej. AG News).  
  - Usa muestreo negativo y ventana de contexto=5.  
  - Extrae los vectores de párrafo y úsalos en una tarea de clasificación temática.  
  - Compara macro‑f1 y micro‑f1 de ambos modelos.

- **Tarea 5.2**: Usa los mismos vectores para un clustering de documentos (k‑means).  
  - Evalúa pureza y NMI (Normalized Mutual Information).  
  - Discute cómo DM (que considera orden) y DBOW (solo bag‑of‑words) afectan al agrupamiento.

#### 6. Fine‑tuning de representaciones para tareas específicas

**Objetivo**: Adaptar embeddings generales a aplicaciones concretas.

- **Tarea 6.1**: Tomando un encoder de documentos (p. ej. un Transformer pre‑entrenado o Doc2Vec),  
  - Añade una capa de clasificación (softmax) y haz fine‑tuning en un dataset de **resumen extractivo**:  
    - Etiquetas binarias por frase (incluir vs. no incluir).  
    - Evalúa ROUGE‑1/2 y F1 binaria.  
  - Experimenta con distintos valores de tasa de aprendizaje $\eta$ y regularización $\lambda$.  

- **Tarea 6.2**: Realiza fine‑tuning para clasificación de sentimiento (p. ej. SST‑2).  
  - Compara desempeño con embeddings estáticos vs. actualizando pesos del encoder completo.  
  - Grafica curvas de validación vs. número de epochs.

#### 7. Aprendizaje multitask

**Objetivo**: Compartir señales de varias tareas para mejorar la generalización.

- **Tarea 7.1**: Diseña un modelo multi‑tarea para **clasificación temática** y **sentimiento** simultáneos:  
  - Encoder compartido + dos cabeceras de clasificación.  
  - Define pérdidas $L_1, L_2$ y pesos $\lambda_1, \lambda_2$.  
  - Entrena en un corpus mixto (por ejemplo, noticias con polaridad).  
  - Ajusta $\lambda$ para equilibrar ambas tareas y documenta el impacto.

- **Tarea 7.2**: Incorpora una tarea auxiliar de **NER** (reconocimiento de entidades) usando un decoder CRF sobre el mismo encoder.  
  - Mide si la señal de NER mejora la clasificación de sentimiento y tema.  
  - Analiza la evolución de las pérdidas individuales y compartida.


#### 8. Métodos de atención supervisada y adapters

**Objetivo**: Explorar mecanismos ligeros de adaptación.

- **Tarea 8.1**: Añade sobre BERT un **módulo de atención supervisada** que aprenda pesos $\alpha_t$ para cada token (ecuaciones de atención vista).  
  - Entrena en un dataset de QA extractiva (SQuAD) y mide Exact Match / F1.  
  - Visualiza los pesos de atención en ejemplos de prueba.

- **Tarea 8.2**: Implementa **Adapters** en un Transformer pre‑entrenado para clasificación de oraciones:  
  - Inserta capas de bajo rango entre capas de BERT.  
  - Entrena únicamente los adapters y la cabecera final.  
  - Compara número de parámetros ajustados vs. fine‑tuning completo y desempeño en SST‑2.


Para cada ejercicio:

1. Presenta un **informe técnico** con:  
   - Descripción del pipeline y arquitectura.  
   - Hiperparámetros utilizados.  
   - Métricas y gráficas de desempeño.  
   - Análisis de ventajas, limitaciones y lecciones aprendidas.

2. **Discusión comparativa** que contraste al menos dos métodos vistos.

3. **Recomendaciones** sobre elección de modelo según requisitos de latencia, datos y recursos.



In [ ]:
## Tus respuestas